In [63]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import polars as pl
from sqlalchemy import create_engine,text
from scipy import stats
import plotly.express as px
import toml
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import h5py

%matplotlib inline
from IPython.display import display, HTML

relative_path = "../../../.."

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:0,.0f}'.format

## Transit Boardings

In [64]:

df = pd.read_csv(os.path.join(relative_path,'outputs','transit','daily_boardings_by_agency.csv'),index_col=0)
df.loc['Region Total','boardings'] = df['boardings'].sum()
df = df.reset_index()
df.rename(columns={'agency_name': 'Agency', 'boardings': 'Boardings'}, inplace=True)
HTML(df.to_html(index=False))

Agency,Boardings
King County Metro,"802,085"
Sound Transit,"540,922"
Community Transit,"95,682"
Pierce Transit,"71,893"
Kitsap Transit,"38,644"
Washington Ferries,"21,517"
Everett Transit,"18,288"
Region Total,"1,589,030"


In [65]:
df = pd.read_csv(os.path.join(relative_path,'outputs','transit','transit_line_results.csv'),index_col=0)

transit_type_map = {
    1: 'Local Bus',
    2: 'Express Bus',
    3: 'Bus Rapid Transit',
    4: 'Streetcar',
    5: 'Commuter Rail',
    6: 'Light Rail',
    7: 'Auto Ferry',
    8: 'Passenger Ferry'
}
df['transit_type'] = df['transit_type'].map(transit_type_map)
df = df.groupby('transit_type').sum()[['boardings']]
df.loc['Total'] = df.sum(axis=0)
df = df.reset_index()
df.rename(columns={'boardings':' Boardings','transit_type':'Transit Type'}, inplace=True)
HTML(df.to_html(index=False))

Transit Type,Boardings
Auto Ferry,"34,341"
Bus Rapid Transit,"396,583"
Commuter Rail,"9,381"
Light Rail,"501,681"
Local Bus,"626,226"
Passenger Ferry,"9,873"
Streetcar,"10,944"
Total,"1,589,030"


In [66]:
df = pd.read_csv(os.path.join(relative_path,'outputs','transit','transit_line_results.csv'),index_col=0)
df = df[(df['agency_code']==6)].groupby('transit_type').sum()[['boardings']]
df.index = df.index.map({1: 'Bus', 3: 'BRT (Stride)', 5: 'Commuter Rail', 6: 'Light Rail'})
df

,boardings
transit_type,
Bus,"7,797"
BRT (Stride),"22,063"
Commuter Rail,"9,381"
Light Rail,"501,681"


In [67]:
# all transit stop locations
df_transit_stops = pd.read_csv(os.path.join(relative_path,'inputs','scenario','networks','transit_stops.csv'), usecols = ['PSRCJunctID', 'x', 'y','light_rail'])

# Load station names
conn = create_engine('sqlite:///../../../../inputs/db/'+config['db_name'])
df_lr_station_names = pd.read_sql( "SELECT * FROM light_rail_stations_2050", con=conn)

# Merge station names to df 
light_rail_stations = df_transit_stops[df_transit_stops['light_rail']==1].\
    merge(df_lr_station_names, left_on=['PSRCJunctID'], right_on=['Station Node'])

In [68]:
# Light rail boardings by station
df_boardings = pd.read_csv(os.path.join(relative_path,'outputs','transit','boardings_by_stop.csv'))
df = pd.merge(df_boardings, light_rail_stations, left_on=['i_node'], right_on=['PSRCJunctID'])

df = df[['total_boardings','Station Name','PSRCJunctID']]#.drop_duplicates()

# pivot table of boardings by station 
df[['PSRCJunctID','Station Name','total_boardings']]

,PSRCJunctID,Station Name,total_boardings
0,197649,Husky Stadium,"12,535"
1,198007,Westlake,"39,810"
2,198009,Symphony,"20,714"
3,198011,Pioneer Square,"18,983"
4,198013,Int'l District,"29,155"
5,198014,Capitol Hill,"18,772"
6,198015,Stadium,"2,507"
7,198016,SODO,"12,504"
8,198017,Beacon Hill,"3,275"
9,198018,Mount Baker,"3,517"


In [69]:
# Light rail boardings by segment
df = pd.read_csv(os.path.join(relative_path,'outputs/transit/transit_line_results.csv'), usecols = ['line_id','mode','description'])
df_light_rail = df[(df['mode']=="r") & (df['description']!="Streetcar-South Lake")].copy().\
    drop_duplicates()
df_light_rail['description'] = df_light_rail['description'].str[:6]
df_light_rail.rename(columns={'description':'Line'}, inplace=True)

In [70]:
# Light rail boardings by segment
df_boardings = pd.read_csv(os.path.join(relative_path,'outputs','transit','transit_segment_results.csv'))

# get line and depart station names
df = df_boardings.merge(df_lr_station_names, left_on=['i_node'], right_on=['Station Node']).\
    merge(df_light_rail, on=['line_id'])

# all station boardings by line
board_station_line = df.groupby(['Line','Station Name'])['segment_boarding'].sum().reset_index()

In [71]:
board_station_line[board_station_line['Line']=='1 Line']

,Line,Station Name,segment_boarding
0,1 Line,Angle Lake,"4,200"
1,1 Line,Ballard Station,"6,067"
2,1 Line,Beacon Hill,"3,275"
3,1 Line,Boeing Access Station,16
4,1 Line,Columbia City,"2,225"
5,1 Line,Denny Station,"6,804"
6,1 Line,East Tacoma Station,"2,470"
7,1 Line,Federal Way Station,"8,211"
8,1 Line,Fife Station,"2,633"
9,1 Line,Graham Station,"1,936"


In [72]:
board_station_line[board_station_line['Line']=='2 Line']

,Line,Station Name,segment_boarding
28,2 Line,Alderwood Station,"2,907"
29,2 Line,Ash Way Station,"2,896"
30,2 Line,BelRed Station,"5,097"
31,2 Line,Bellevue Downtown Station,"13,661"
32,2 Line,Capitol Hill,"9,702"
33,2 Line,Downtown Redmond Station,"5,858"
34,2 Line,East Main Station,"8,860"
35,2 Line,Husky Stadium,"5,945"
36,2 Line,Int'l District,"11,759"
37,2 Line,Judkins Park Station,"3,337"


In [73]:
board_station_line[board_station_line['Line']=='3 Line']

,Line,Station Name,segment_boarding
57,3 Line,Alderwood Station,"3,626"
58,3 Line,Ash Way Station,"4,054"
59,3 Line,Avalon Station,"2,731"
60,3 Line,Capitol Hill,"9,069"
61,3 Line,Delridge Station,"6,633"
62,3 Line,Everett Station,"6,660"
63,3 Line,Husky Stadium,"6,590"
64,3 Line,Int'l District,"6,840"
65,3 Line,Lynnwood Station,"6,084"
66,3 Line,Mariner P&R Station,"4,356"


In [74]:
board_station_line[board_station_line['Line']=='4 Line']

,Line,Station Name,segment_boarding
82,4 Line,Bellevue Downtown Station,"4,466"
83,4 Line,East Main Station,"4,885"
84,4 Line,Eastgate Station,"2,771"
85,4 Line,Issaquah Station,"3,034"
86,4 Line,Richards Rd Station,"2,683"
87,4 Line,South Kirkland Station,794
88,4 Line,Wilburton Station,"1,319"


In [75]:
board_station_line[board_station_line['Line']=='T Line']

,Line,Station Name,segment_boarding
89,T Line,6th Ave Station,"1,907"
90,T Line,Ainsworth Station,484
91,T Line,Commerce Station,486
92,T Line,Convention Center Station,317
93,T Line,Hilltop District Station,499
94,T Line,Old City Hall Station,565
95,T Line,Pearl Station,"1,091"
96,T Line,S 25th St Station,684
97,T Line,S 4th St Station,0
98,T Line,Sprague Station,258


In [76]:
df = pd.read_csv(os.path.join(relative_path,'outputs','transit','total_transit_trips.csv'),index_col=0)
df.rename(index={'commuter_rail': 'Commuter Rail',
                'litrat': 'Light Rail',
                'ferry': 'Auto Ferry',
                'passenger_ferry': 'Passenger Ferry',
                'trnst': 'Bus'}, inplace=True)
df.columns = ['Total Trips']
df.loc['Total'] = df.sum(axis=0)
df.reset_index(inplace=True)
df.rename(columns={'index': 'Transit Type'}, inplace=True)
HTML(df.to_html(index=False))

Transit Type,Total Trips
Commuter Rail,"10,362"
Light Rail,"466,856"
Auto Ferry,"37,286"
Passenger Ferry,"4,493"
Bus,"562,802"
Total,"1,081,798"


## Households Near HCT

In [77]:
# List of Stations
# Load transit stops file

#"The definition we have been using for the RTP is BRT, LRT, Commuter Rail, StreetCar and Ferry."
df = pd.read_csv(r'../../../../inputs/scenario/networks/transit_stops.csv')
# Streetcar is coded as light rail
df['hct'] = 0
df.loc[df[['commuter_rail','light_rail','ferry','brt']].sum(axis=1) > 0,'hct'] = 1
df_hct = df[df['hct'] == 1]

# Map of Stations
# Load as a geodataframe
gdf_hct = gpd.GeoDataFrame(
    df_hct, geometry=gpd.points_from_xy(df_hct.x, df_hct.y))

gdf_hct.crs = 'EPSG:2285'

In [78]:
df_lu = pl.read_csv(r'..\..\..\..\inputs\scenario\landuse\parcels_urbansim.txt', separator=" ").to_pandas()

# Load as a geodataframe
gdf_lu = gpd.GeoDataFrame(
    df_lu, geometry=gpd.points_from_xy(df_lu.xcoord_p, df_lu.ycoord_p))

gdf_lu.crs = 'EPSG:2285'

In [79]:
parcel_geog = pd.read_sql_table('parcel_'+config['base_year']+'_geography', 'sqlite:///../../../../inputs/db/'+config['db_name'])

In [80]:
df_lu = df_lu.merge(parcel_geog, left_on='parcelid', right_on='ParcelID', how='left')
# Add a field that defines whether a parcel is inside (1) or outside (0) any RGC
df_lu['RGC_binary'] = 0
df_lu.loc[df_lu['GrowthCenterName'] != 'Not in RGC', 'RGC_binary'] = 1

In [81]:
def calculate_buffer(gdf_lu, _gdf_hct, distance):
    
    # Buffer the HCT station gdf
    _gdf_hct['geometry'] = _gdf_hct.buffer(distance)

    gdf_intersect = gpd.overlay(_gdf_hct, gdf_lu, how="intersection", keep_geom_type=False)
    df = df_lu[df_lu['parcelid'].isin(gdf_intersect['parcelid'].unique())]
    
    return df, gdf_intersect

def aggregate_parcels(df, col_dict, sum_field):
    results_df = pd.DataFrame()
    for col, name in col_dict.items():
        _df = df[[col,sum_field]].groupby(col).sum()[[sum_field]]
        _df['group'] = name
        results_df = pd.concat([results_df,_df])
    results_df = results_df.reset_index()
    
    return results_df

In [82]:
df_025, gdf_025 = calculate_buffer(gdf_lu, gdf_hct.copy(), distance=5280.0/4)

In [83]:
df_050, gdf_050 = calculate_buffer(gdf_lu, gdf_hct.copy(), distance=5280.0/2)

In [84]:
pd.options.display.float_format = '{:0,.0f}'.format
col_dict = {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
            'equity_focus_areas_2023__efa_dis': 'Disability',
           'rg_proposed': 'Regional Geography',
            'CountyName': 'County',
            'GrowthCenterName': 'Regional Growth Center',
            'RGC_binary': 'RGC Binary',
            'Region': 'Region'
           }

hct_hh_df = pd.DataFrame()
df_025 = df_025.copy()
df_025['Region'] = 1
df = aggregate_parcels(df_025, col_dict, 'hh_p')
df['distance'] = 0.25
hct_hh_df = pd.concat([hct_hh_df,df])
df_050 = df_050.copy()
df_050['Region'] = 1
df = aggregate_parcels(df_050, col_dict, 'hh_p')
df['distance'] = 0.50
hct_hh_df = pd.concat([hct_hh_df,df])

In [85]:
pd.options.display.float_format = '{:0,.0f}'.format

In [86]:

df = hct_hh_df[hct_hh_df['group'] == 'Region']
df = df.rename(columns={'hh_p': 'Total'}).copy()
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Region'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df = df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'})

df['% of Households (1/4 Mile)'] = df['1/4 Mile']/df_lu['hh_p'].sum()
df['% of Households (1/2 Mile)'] = df['1/2 Mile']/df_lu['hh_p'].sum()

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Region,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
1,"785,293","1,144,663",32.4%,47.3%


In [87]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'County'].copy()
df = df.rename(columns={'hh_p': 'Total'}).copy()
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'County'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

_df = df.merge(df_lu[['CountyName','hh_p']].groupby('CountyName').sum()[['hh_p']].reset_index(),left_on='County', right_on='CountyName')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['hh_p']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['hh_p']

df = df.merge(_df[['County', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='County')

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

County,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
King,"618,389","826,752",48.0%,64.2%
Kitsap,"17,724","36,500",11.8%,24.3%
Pierce,"45,416","77,561",9.1%,15.5%
Snohomish,"103,764","203,850",21.5%,42.3%


In [88]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'RGC Binary'].copy()
df.rename(columns={'hh_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'RGC Binary'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)


_df = df.merge(df_lu[['RGC_binary','hh_p']].groupby('RGC_binary').sum()[['hh_p']].reset_index(),left_on='RGC Binary', 
               right_on='RGC_binary')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['hh_p']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['hh_p']

df = df.merge(_df[['RGC Binary', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='RGC Binary')
df['RGC Designation'] = df['RGC Binary'].map({0: 'Outside RGC', 1: 'Inside RGC'})
df = df[['RGC Designation','1/4 Mile','1/2 Mile','% of Households (1/4 Mile)', '% of Households (1/2 Mile)']]
output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

RGC Designation,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Outside RGC,"440,981","739,047",22.3%,37.4%
Inside RGC,"344,312","405,616",77.2%,91.0%


In [89]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'Regional Growth Center'].copy()
df.rename(columns={'hh_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Growth Center'
# Ensure all RGCs are included, even those that have no HCT access
for center in parcel_geog['GrowthCenterName'].unique():
    if center not in df.index:
        df.loc[center] = 0

df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

# Calcualte the percent of households in this geography within 1/4 and 1/2 mile
_df = df.merge(df_lu[['GrowthCenterName','hh_p']].groupby('GrowthCenterName').sum()[['hh_p']].reset_index(),left_on='Regional Growth Center', 
               right_on='GrowthCenterName')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['hh_p']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['hh_p']

df = df.merge(_df[['Regional Growth Center', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='Regional Growth Center')

df = df.sort_values('Regional Growth Center')

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Growth Center,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Auburn,"4,805","5,005",96.0%,100.0%
Bellevue,"32,173","32,904",97.8%,100.0%
Bothell Canyon Park,729,"3,438",13.7%,64.7%
Bremerton,"5,360","5,360",99.5%,99.5%
Burien,"8,570","8,803",97.4%,100.0%
Everett,"11,135","20,877",49.7%,93.2%
Federal Way,"5,949","6,226",95.6%,100.0%
Greater Downtown Kirkland,"5,468","6,157",88.7%,99.9%
Issaquah,"1,196","2,634",45.4%,100.0%
Kent,"3,504","3,854",90.9%,100.0%


In [90]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'Regional Geography'].copy()
df.rename(columns={'hh_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Geography'
df.reset_index(inplace=True)

df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
HTML(df.to_html(index=False))

_df = df.merge(df_lu[['rg_proposed','hh_p']].groupby('rg_proposed').sum()[['hh_p']].reset_index(),left_on='Regional Geography', 
               right_on='rg_proposed')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['hh_p']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['hh_p']

df = df.merge(_df[['Regional Geography', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='Regional Geography')

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Geography,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Cities and Towns,464,"2,068",0.3%,1.2%
Core Cities,"188,848","296,307",33.5%,52.5%
High Capacity Transit Communities,"105,045","187,380",20.4%,36.4%
Metropolitan Cities,"486,131","645,044",56.9%,75.5%
Rural Areas,"1,198","3,406",0.5%,1.5%
Urban Unincorporated Areas,"3,607","10,458",4.3%,12.6%


In [91]:
pd.options.display.float_format = '{:0,.2f}'.format
# df = hct_hh_df.copy()
# df = df[df['index'] == 1]
# df.rename(columns={'hh_p': 'Total'}, inplace=True)
# df.drop('index', axis=1, inplace=True)
# df = df.pivot_table(index='group', columns='distance', values='Total', aggfunc='sum')
# df[0.25] = df[0.25].astype('float32')
# df[0.5] = df[0.5].astype('float32')
# df.index.name = 'Equity Geography'
# df = df.reset_index()
# df = df.rename_axis(None, axis=1)
# df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

In [92]:
pd.options.display.float_format = '{:0,.1f}'.format

equity_dict = {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
            'equity_focus_areas_2023__efa_dis': 'Disability'}

df = hct_hh_df[hct_hh_df['group'].isin(equity_dict.values())].copy()
df['EFA Type'] = df['index'].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })

df = df.pivot_table(index=['group','EFA Type'], columns='distance', values='hh_p', aggfunc='sum').reset_index()

# Merge with total households to get shares of group
efa_pop_tot = df_lu[equity_dict.keys()].sum().to_dict()

for lu_col, label in equity_dict.items():
    df.loc[df['group'] == label, "Total Pop"] = efa_pop_tot[lu_col]
df['% of Households (1/4 Mile)'] = df[0.25]/df['Total Pop']
df['% of Households (1/2 Mile)'] = df[0.5]/df['Total Pop']

df.rename(columns={'group': 'Group', 0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df = df.rename_axis(None, axis=1)
df.drop('Total Pop', inplace=True, axis=1)
output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Group,EFA Type,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Disability,Above Regional Average,227613,340182,28.8%,43.1%
Disability,Below Regional Average,393125,573643,49.8%,72.6%
Disability,Higher Share of Equity Population,164555,230838,20.8%,29.2%
LEP,Above Regional Average,205826,300161,35.7%,52.0%
LEP,Below Regional Average,373532,544456,64.7%,94.4%
LEP,Higher Share of Equity Population,205935,300046,35.7%,52.0%
Older,Above Regional Average,180295,280048,19.3%,30.0%
Older,Below Regional Average,506764,726806,54.2%,77.8%
Older,Higher Share of Equity Population,98234,137809,10.5%,14.7%
People of Color,Above Regional Average,287304,416964,45.2%,65.6%


## Jobs Near HCT

In [93]:
pd.options.display.float_format = '{:0,.0f}'.format
col_dict = {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
            'equity_focus_areas_2023__efa_dis': 'Disability',
           'rg_proposed': 'Regional Geography',
            'CountyName': 'County',
            'GrowthCenterName': 'Regional Growth Center',
            'RGC_binary': 'RGC Binary',
            'Region': 'Region'
           }

hct_hh_df = pd.DataFrame()
df_025['Region'] = 1
df = aggregate_parcels(df_025, col_dict, 'emptot_p')
df['distance'] = 0.25
hct_hh_df = pd.concat([hct_hh_df,df])
df_050['Region'] = 1
df = aggregate_parcels(df_050, col_dict, 'emptot_p')
df['distance'] = 0.50
hct_hh_df = pd.concat([hct_hh_df,df])

In [94]:

df = hct_hh_df[hct_hh_df['group'] == 'Region'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Region'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

df['% of Jobs (1/4 Mile)'] = df['1/4 Mile']/df_lu['emptot_p'].sum()
df['% of Jobs (1/2 Mile)'] = df['1/2 Mile']/df_lu['emptot_p'].sum()

output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Region,1/4 Mile,1/2 Mile,% of Jobs (1/4 Mile),% of Jobs (1/2 Mile)
1,"1,620,813","2,120,424",51.1%,66.8%


In [95]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'County'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'County'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

_df = df.merge(df_lu[['CountyName','emptot_p']].groupby('CountyName').sum()[['emptot_p']].reset_index(),left_on='County', right_on='CountyName')
_df['% of Jobs (1/4 Mile)'] = _df['1/4 Mile']/_df['emptot_p']
_df['% of Jobs (1/2 Mile)'] = _df['1/2 Mile']/_df['emptot_p']

df = df.merge(_df[['County', '% of Jobs (1/4 Mile)', '% of Jobs (1/2 Mile)']], 
              on='County')

output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

County,1/4 Mile,1/2 Mile,% of Jobs (1/4 Mile),% of Jobs (1/2 Mile)
King,"1,314,773","1,616,180",64.2%,78.9%
Kitsap,"40,863","70,977",27.8%,48.2%
Pierce,"100,019","145,869",20.6%,30.0%
Snohomish,"165,158","287,398",33.5%,58.3%


In [96]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'RGC Binary'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'RGC Binary'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)


_df = df.merge(df_lu[['RGC_binary','emptot_p']].groupby('RGC_binary').sum()[['emptot_p']].reset_index(),left_on='RGC Binary', 
               right_on='RGC_binary')
_df['% of Jobs (1/4 Mile)'] = _df['1/4 Mile']/_df['emptot_p']
_df['% of Jobs (1/2 Mile)'] = _df['1/2 Mile']/_df['emptot_p']

df = df.merge(_df[['RGC Binary', '% of Jobs (1/4 Mile)', '% of Jobs (1/2 Mile)']], 
              on='RGC Binary')
df['RGC Designation'] = df['RGC Binary'].map({0: 'Outside RGC', 1: 'Inside RGC'})
df = df[['RGC Designation','1/4 Mile','1/2 Mile','% of Jobs (1/4 Mile)', '% of Jobs (1/2 Mile)']]
output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

RGC Designation,1/4 Mile,1/2 Mile,% of Jobs (1/4 Mile),% of Jobs (1/2 Mile)
Outside RGC,"639,997","1,000,980",32.6%,51.0%
Inside RGC,"980,816","1,119,444",81.1%,92.5%


In [97]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'Regional Growth Center'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Growth Center'
# Ensure all RGCs are included, even those that have no HCT access
for center in parcel_geog['GrowthCenterName'].unique():
    if center not in df.index:
        df.loc[center] = 0
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df['% of Total Jobs (1/4 Mile)'] = df['1/4 Mile']/df_lu['emptot_p'].sum()
df['% of Total Jobs (1/2 Mile)'] = df['1/2 Mile']/df_lu['emptot_p'].sum()

output = df.to_html(formatters={
    '% of Total Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Total Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Growth Center,1/4 Mile,1/2 Mile,% of Total Jobs (1/4 Mile),% of Total Jobs (1/2 Mile)
Auburn,"9,039","9,693",0.3%,0.3%
Bellevue,"97,138","98,694",3.1%,3.1%
Bothell Canyon Park,"3,600","11,345",0.1%,0.4%
Bremerton,"8,865","17,996",0.3%,0.6%
Burien,"8,753","8,962",0.3%,0.3%
Everett,"36,484","51,927",1.1%,1.6%
Federal Way,"11,815","12,628",0.4%,0.4%
Greater Downtown Kirkland,"19,743","20,446",0.6%,0.6%
Issaquah,"3,217","14,131",0.1%,0.4%
Kent,"31,480","33,419",1.0%,1.1%


In [98]:
df = hct_hh_df[hct_hh_df['group'] == 'Regional Geography'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Geography'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df['% of Total Jobs (1/4 Mile)'] = df['1/4 Mile']/df_lu['emptot_p'].sum()
df['% of Total Jobs (1/2 Mile)'] = df['1/2 Mile']/df_lu['emptot_p'].sum()

output = df.to_html(formatters={
    '% of Total Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Total Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Geography,1/4 Mile,1/2 Mile,% of Total Jobs (1/4 Mile),% of Total Jobs (1/2 Mile)
Cities and Towns,847,"2,414",0.0%,0.1%
Core Cities,"468,450","676,318",14.8%,21.3%
High Capacity Transit Communities,"105,878","165,397",3.3%,5.2%
Metropolitan Cities,"1,033,835","1,255,427",32.6%,39.5%
Rural Areas,"4,474","7,749",0.1%,0.2%
Urban Unincorporated Areas,"7,329","13,119",0.2%,0.4%


In [99]:
pd.options.display.float_format = '{:0,.0f}'.format

equity_dict = {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
            'equity_focus_areas_2023__efa_dis': 'Disability'}

df = hct_hh_df[hct_hh_df['group'].isin(equity_dict.values())].copy()
df['EFA Type'] = df['index'].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })

df = df.pivot_table(index=['group','EFA Type'], columns='distance', values='emptot_p', aggfunc='sum').reset_index()

# Merge with total households to get shares of group
efa_pop_tot = df_lu[equity_dict.keys()].sum().to_dict()

df[0.25] = df[0.25].astype("float")
df[0.50] = df[0.50].astype("float")

for lu_col, label in equity_dict.items():
    df.loc[df['group'] == label, "Total Jobs"] = efa_pop_tot[lu_col]
df['% of Households (1/4 Mile)'] = df[0.25]/df['Total Jobs']
df['% of Households (1/2 Mile)'] = df[0.5]/df['Total Jobs']


df.rename(columns={'group': 'Group', 0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df = df.rename_axis(None, axis=1)
df.drop('Total Jobs', inplace=True, axis=1)
output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Group,EFA Type,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Disability,Above Regional Average,"430,049","570,038",1,1
Disability,Below Regional Average,"740,816","995,339",1,1
Disability,Higher Share of Equity Population,"449,948","555,047",1,1
LEP,Above Regional Average,"460,269","590,890",1,1
LEP,Below Regional Average,"726,755","962,658",1,2
LEP,Higher Share of Equity Population,"433,789","566,876",1,1
Older,Above Regional Average,"400,354","524,902",0,1
Older,Below Regional Average,"1,045,955","1,355,148",1,1
Older,Higher Share of Equity Population,"174,504","240,374",0,0
People of Color,Above Regional Average,"685,321","847,282",1,1


## Transit Access Mode
Percent of Transit Trips Accessed by Walking

In [100]:
pd.options.display.float_format = '{:0,.1%}'.format
df = pd.read_csv(r'../../../../outputs/agg/dash/trip_mode_by_tour_mode.csv')
_df = df.copy()
pnr_transit_trips = _df[(_df['tmodetp'] == 'Park') & (_df['mode'] == 'Transit')]['trexpfac'].sum()
walk_transit_trips = _df[(_df['tmodetp'] == 'Transit') & (_df['mode'] == 'Transit')]['trexpfac'].sum()
reg_access = (walk_transit_trips/(walk_transit_trips+pnr_transit_trips))
pd.DataFrame([reg_access], columns=['% Transit trips Accessed by Walking'], index=['Region'])


,% Transit trips Accessed by Walking
Region,96.3%


In [101]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = df.copy()
results_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']
# for col in ['hh_disability','hh_elderly','hh_english','hh_poverty','hh_racial','hh_youth']:
for col in summary_config["hh_equity_geogs"]:
    pnr_transit_trips = _df[(_df[col] == 1) & 
                            (_df['tmodetp'] == 'Park') &
                            (_df['mode'] == 'Transit')]['trexpfac'].sum()
    walk_transit_trips = _df[(_df[col] == 1) & 
                                (_df['tmodetp'] == 'Transit') & 
                                (_df['mode'] == 'Transit')]['trexpfac'].sum()
    if (pnr_transit_trips > 0) & (walk_transit_trips > 0):
        results_df.loc[col,'% Transit Trips Accessed by Walking'] = walk_transit_trips/(walk_transit_trips+pnr_transit_trips)
# results_df.rename(columns={'_reg':'> Regional Average', '_50': '> 50%'}, inplace=True)
results_df

,% Transit Trips Accessed by Walking
hh_efa_dis,96.5%
hh_efa_older,95.8%
hh_efa_lep,96.3%
hh_efa_pov200,96.7%
hh_efa_poc,96.4%
hh_efa_youth,94.8%


## Transit Equity

### Trip purpose by transit mode type

In [102]:


myh5 = h5py.File(r'..\..\..\..\inputs\scenario\landuse\hh_and_persons.h5', 'r')
person = pd.read_csv(r'..\..\..\..\outputs\daysim\_person.tsv', sep='\t')
df_trip = pd.read_csv(r'..\..\..\..\outputs\daysim\_trip.tsv', sep='\t')
df_hh = pd.read_csv(r'..\..\..\..\outputs\daysim\_household.tsv', sep='\t')

In [103]:
# Create a DataFrame from the h5 file
df_person = pd.DataFrame()
for col in ['hhno','pno','prace']:
    df_person[col] = myh5['Person'][col][:]

df_trip = df_trip.merge(df_person[['hhno','pno','prace']], on=['hhno','pno'], how='left')

In [104]:
# Create group of work, school, and all other purposes
df_trip['Purpose Type'] = 'Other'
df_trip.loc[df_trip['dpurp']==0, 'Purpose Type'] = 'Home'
df_trip.loc[df_trip['dpurp']==1, 'Purpose Type'] = 'Work'
df_trip.loc[df_trip['dpurp']==2, 'Purpose Type'] = 'School'

# Rename path types
df_trip.loc[df_trip['pathtype']==1, 'Path Type'] = 'Drive'
df_trip.loc[df_trip['pathtype']==3, 'Path Type'] = 'Bus'
df_trip.loc[df_trip['pathtype']==4, 'Path Type'] = 'Light Rail'
df_trip.loc[df_trip['pathtype']==5, 'Path Type'] = 'Ferry 1'
df_trip.loc[df_trip['pathtype']==6, 'Path Type'] = 'Commuter Rail'
df_trip.loc[df_trip['pathtype']==7, 'Path Type'] = 'Ferry 2'

df = pd.pivot_table(df_trip[(df_trip['dpurp']!=0) & (~df_trip['pathtype'].isin([0,'Drive']))], values='trexpfac', index='Path Type', columns='Purpose Type', aggfunc='sum')

df['Total'] = df.sum(axis=1)
for row in df.index:
    for col in ['Work','School','Other']:
        df.loc[row, col + ' %'] = df.loc[row, col] / df.loc[row, 'Total']

df[['Work %','School %','Other %']] = df[['Work %','School %','Other %']].map("{:,.1%}".format)
df[['Work %','School %','Other %']]

Purpose Type,Work %,School %,Other %
Path Type,,,
Bus,15.1%,12.9%,71.9%
Commuter Rail,50.9%,2.0%,47.1%
Drive,20.6%,5.5%,73.9%
Ferry 1,66.2%,6.4%,27.4%
Ferry 2,51.7%,9.5%,38.8%
Light Rail,31.3%,12.6%,56.1%


In [105]:

race_dict = {
    '1': 'White alone',
    '2': 'Black or African American alone',
    '3': 'American Indian alone',
    '4': 'Alaska Native alone',
    '5': 'American Indian and Alaska Native tribes specified',
    '6': 'Asian alone',
    '7': 'Native Hawaiian and Other Pacific Islander alone',
    '8': 'Some Other Race alone',
    '9': 'Two or More Races'
}


df_trip['Race'] = df_trip['prace'].astype('int').astype('str').map(race_dict)

### Transit type distribution by race of rider

In [106]:
# Submode Ridership by race
df = pd.pivot_table(df_trip[(df_trip['pathtype']!="Drive")], values='trexpfac', index='Path Type', columns='Race', aggfunc="sum")
race_cols = ['White alone','Black or African American alone','American Indian alone',
             'Alaska Native alone','American Indian and Alaska Native tribes specified','Asian alone',
             'Native Hawaiian and Other Pacific Islander alone','Some Other Race alone','Two or More Races']
# Calculate shares of total by Race for each Path Type
df['Total'] = df.sum(axis=1)
df = df.astype('float')
for row in df.index:
    for col in race_cols:
        df.loc[row, col] = df.loc[row, col] / df.loc[row, 'Total']
df[race_cols] = df[race_cols].map("{:,.1%}".format)
df[race_cols]

Race,White alone,Black or African American alone,American Indian alone,Alaska Native alone,American Indian and Alaska Native tribes specified,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,Two or More Races
Path Type,,,,,,,,,
Bus,63.2%,6.7%,0.6%,0.1%,0.1%,15.7%,0.7%,4.0%,8.8%
Commuter Rail,65.4%,7.2%,0.8%,0.1%,0.1%,12.0%,1.3%,4.9%,8.1%
Drive,65.2%,5.7%,0.7%,0.1%,0.1%,14.3%,0.9%,4.3%,8.8%
Ferry 1,69.6%,4.9%,0.7%,0.1%,0.1%,13.6%,0.4%,3.4%,7.1%
Ferry 2,71.6%,4.1%,0.7%,0.0%,0.1%,10.8%,0.7%,3.6%,8.3%
Light Rail,62.3%,6.4%,0.6%,0.1%,0.1%,17.3%,0.7%,4.0%,8.6%


In [107]:
df_trip = df_trip.merge(df_hh[['hhno','hhincome']], on=['hhno'], how='left')

### Income distribution by household income of rider

In [108]:
# Create bins for hhincome
bins = [0, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000, 200000, np.inf]
labels = ['<20k', '20-40k', '40-60k', '60-80k', '80-100k', '100-120k', '120-140k', '140-160k', '160-180k', '180-200k', '>200k']
df_trip['Income Bin'] = pd.cut(df_trip['hhincome'], bins=bins, labels=labels, right=False)
df_trip['Income Bin'] = df_trip['Income Bin'].astype('str')

# Transit Submodes by Income
df = pd.pivot_table(df_trip[(df_trip['Path Type']!="Drive")], values='trexpfac', index='Path Type', columns='Income Bin', aggfunc="sum")

# display labels with no significant figs
df[labels] = df[labels].map("{:0,.0f}".format)
df[labels]

Income Bin,<20k,20-40k,40-60k,60-80k,80-100k,100-120k,120-140k,140-160k,160-180k,180-200k,>200k
Path Type,,,,,,,,,,,
Bus,"61,699","59,772","55,707","49,256","48,288","39,557","36,334","31,654","25,929","25,222","127,634"
Commuter Rail,266,464,777,911,"1,269",991,982,790,628,600,"2,486"
Ferry 1,149,243,286,352,362,295,287,307,288,242,"1,660"
Ferry 2,"1,426","1,997","2,864","3,553","3,515","3,209","2,914","2,752","2,141","2,073","9,975"
Light Rail,"33,287","36,009","38,668","37,633","39,082","32,995","31,199","28,260","23,531","22,972","120,412"


In [109]:
pd.pivot_table(
    df_trip[df_trip['Path Type'] != "Drive"],
    values='hhincome',
    index='Path Type',
    aggfunc="median"
).astype(int).map(lambda x: "${:,.0f}".format(x))

,hhincome
Path Type,
Bus,"$103,172"
Commuter Rail,"$127,040"
Ferry 1,"$157,102"
Ferry 2,"$130,855"
Light Rail,"$122,569"
